## ***XGBoost Regressor***

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("C:\\Users\\default.DESKTOP-GAN0M7C\\my_projects\\data\\amazon data\\Amazon Sale Report.csv")

C:\Users\default.DESKTOP-GAN0M7C\AppData\Local\Temp\ipykernel_1748\179680575.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:\\Users\\default.DESKTOP-GAN0M7C\\my_projects\\data\\amazon data\\Amazon Sale Report.csv")


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

In [5]:

def preprocess_orders(data: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the orders dataset:
    - Drop irrelevant or high-cardinality columns
    - Rename columns if needed
    - Extract useful features from 'Date'
    - Create derived features like day, month, weekday, weekend flag
    """
    
    # 1. Drop unwanted columns if they exist
    DROP_COLS = [
        "index",
        "Order ID",           # pure identifier
        "SKU",                # high cardinality ID
        "promotion-ids",      # sparse / leakage-prone
        "Unnamed: 22",
        "ASIN",
        "fulfilled-by",
        "ship-postal-code",
        "currency",
        "Size"
    ]
    
    data = data.drop(columns=[c for c in DROP_COLS if c in data.columns])
    
    # 2. Rename columns if necessary
    if 'Sales Channel ' in data.columns:
        data.rename(columns={'Sales Channel ':'Sales Channel'}, inplace=True)
    
    # 3. Convert 'Date' to datetime safely
    if 'Date' in data.columns:
        data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")
        
        # Extract date-related features
        data["order_day"] = data["Date"].dt.day_name()
        data["order_month"] = data["Date"].dt.month_name()
        data["order_weekday"] = data["Date"].dt.weekday
        data["is_weekend"] = data["order_weekday"].isin([5,6]).astype(int)
        
        # Drop original Date column
        data = data.drop(columns=["Date"])
    
    return data


In [6]:
data = preprocess_orders(data)
print(data.head())

C:\Users\default.DESKTOP-GAN0M7C\AppData\Local\Temp\ipykernel_1748\103245547.py:32: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Date"] = pd.to_datetime(data["Date"], dayfirst=True, errors="coerce")


                         Status Fulfilment Sales Channel ship-service-level  \
0                     Cancelled   Merchant     Amazon.in           Standard   
1  Shipped - Delivered to Buyer   Merchant     Amazon.in           Standard   
2                       Shipped     Amazon     Amazon.in          Expedited   
3                     Cancelled   Merchant     Amazon.in           Standard   
4                       Shipped     Amazon     Amazon.in          Expedited   

     Style       Category Courier Status  Qty  Amount    ship-city  \
0   SET389            Set            NaN    0  647.62       MUMBAI   
1  JNE3781          kurta        Shipped    1  406.00    BENGALURU   
2  JNE3371          kurta        Shipped    1  329.00  NAVI MUMBAI   
3    J0341  Western Dress            NaN    0  753.33   PUDUCHERRY   
4  JNE3671            Top        Shipped    1  574.00      CHENNAI   

    ship-state ship-country    B2B order_day order_month  order_weekday  \
0  MAHARASHTRA           IN  

### Feature Engineering

In [7]:
# seperating numerical and categorical columns
numerical_cols = data.select_dtypes(include="number").columns
print("numerical_cols:\n", numerical_cols)
print()

categorical_cols = data.select_dtypes(exclude="number").columns
print("catgeorical_cols:\n", categorical_cols)

numerical_cols:
 Index(['Qty', 'Amount', 'order_weekday', 'is_weekend'], dtype='object')

catgeorical_cols:
 Index(['Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Style',
       'Category', 'Courier Status', 'ship-city', 'ship-state', 'ship-country',
       'B2B', 'order_day', 'order_month'],
      dtype='object')


In [8]:
numerical_cols = numerical_cols.drop(['Amount'])
numerical_cols

Index(['Qty', 'order_weekday', 'is_weekend'], dtype='object')

In [10]:
!pip install category_encoders

In [11]:
# ml preprocessing pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from category_encoders.count import CountEncoder

def build_tree_preprocessor(num_cols, cat_cols):
    
    num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median"))
        # no scaling for trees
    ])

    cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", CountEncoder(normalize=True))  # frequency encoding
    ])

    return ColumnTransformer([
        ("num", num_pipeline, num_cols),
        ("cat", cat_pipeline, cat_cols)
    ])


In [12]:
# for linear models -> scaling is on!
preprocessor = build_tree_preprocessor(
    num_cols=numerical_cols,
    cat_cols=categorical_cols,
)

In [13]:
preprocessor

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [14]:

# seperating the input and target variable from data
TARGET = "Amount"   # regression target

X = data.drop(columns=[TARGET])
y = data[TARGET]


In [15]:
# fill target with median values
y.fillna(value=data['Amount'].median(), inplace=True)


In [16]:
X.shape

(128975, 16)

In [17]:
y.shape

(128975,)

In [18]:
# seperating the input and target data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


In [19]:
X_train.shape

(103180, 16)

In [20]:
y_train.shape

(103180,)

In [21]:
# apply preprocessor and take the sample records

def process_and_sample(X_train, X_test, y_train, sample_frac=0.1, n_bins=10, preprocessor=None, random_state=42):
    # 1. Fit preprocessor on full X_train
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    
    # 2. Create quantile bins for stratified sampling
    y_bins = pd.qcut(y_train, q=n_bins, duplicates="drop")
    
    # 3. Stratified sampling
    X_train_sampled, _, y_train_sampled, _ = train_test_split(
        X_train_processed,
        y_train,
        test_size=1-sample_frac,   # keep fraction for training
        stratify=y_bins,
        random_state=random_state
    )
    
    return X_train_processed, X_test_processed, X_train_sampled, y_train_sampled


In [22]:
X_train_processed, X_test_processed, X_train_sampled, y_train_sampled = process_and_sample(
    X_train, X_test, y_train, sample_frac=0.25, preprocessor=preprocessor
)

print("Full X_train_processed shape:", X_train_processed.shape)
print("Sampled X_train_sampled shape:", X_train_sampled.shape)


Full X_train_processed shape: (103180, 16)
Sampled X_train_sampled shape: (25795, 16)


### Model Training

In [23]:
!pip install xgboost

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/72.0 MB 3.7 MB/s eta 0:00:20
    --------------------------------------- 1.6/72.0 MB 3.4 MB/s eta 0:00:21
   - -------------------------------------- 1.8/72.0 MB 2.9 MB/s eta 0:00:25
   - -------------------------------------- 2.4/72.0 MB 2.8 MB/s eta 0:00:25
   - -------------------------------------- 2.9/72.0 MB 2.7 MB/s eta 0:00:26
   -- ------------------------------------- 3.9/72.0 MB 2.9 MB/s eta 0:00:24
   -- ------------------------------------- 4.2/72.0 MB 2.9 MB/s eta 0:00:24
   -- ------------------------------------- 4.7/72.0 MB 2.8 MB/s eta 0:00:25
   -- ------------------------------------- 5.2/72.0 MB 2.8 MB/s eta 0:00:25
   --- ------------------------------------ 5.8/72.0 MB 2.7 MB/s eta 0:00:25
   --- ------------------------------------ 6.3/72.0 MB 2.7 MB/s eta 0:00:25
   --- ------

In [24]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [25]:
# utility function to check metrics
def evaluate_regression(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    return {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "R2": r2
    }


In [26]:
xgb_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)


In [27]:
# training the model
xgb_model.fit(
    X_train_sampled,
    y_train_sampled,
    eval_set=[(X_train_sampled, y_train_sampled)],
    verbose=False
)


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [28]:
# evaluate the model
y_pred = xgb_model.predict(X_test_processed)

metrics = evaluate_regression(y_test, y_pred)
metrics


{'MAE': 93.44423299039238,
 'MSE': 23904.826465663893,
 'RMSE': np.float64(154.61185745493097),
 'R2': 0.6842928275080475}

**XGBoost > Gradient Boosting**
→ better regularization, parallelism, handling of complex interactions

**Sampled training is correct**
→ for experimentation, benchmarking, model selection

**Final production model**
→ retrain XGBoost on full processed training data

now, finding the best parameters for the XGBoost model using ***RandomSearchCV***

XGBoost has many interacting hyperparameters

GridSearch becomes too slow

RandomizedSearch finds near-optimal params efficiently

In [30]:

xgb_base = XGBRegressor(
    objective="reg:squarederror",
    n_jobs=-1,
    random_state=42
)


In [31]:
param_dist = {
    "n_estimators": [300, 500, 800, 1200],
    "max_depth": [3, 4, 5, 6, 8],
    "learning_rate": [0.01, 0.03, 0.05, 0.1],
    "subsample": [0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.6, 0.7, 0.8, 0.9, 1.0],
    "min_child_weight": [1, 3, 5, 10],
    "gamma": [0, 0.1, 0.3, 0.5],
    "reg_alpha": [0, 0.1, 0.5, 1.0],   # L1
    "reg_lambda": [1.0, 1.5, 2.0, 3.0] # L2
}


In [32]:
# train with randomsearchcv
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator=xgb_base,
    param_distributions=param_dist,
    n_iter=40,              # ~40–60 is ideal
    scoring="neg_mean_absolute_error",
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train_sampled, y_train_sampled)


Fitting 3 folds for each of 40 candidates, totalling 120 fits


,estimator,"XGBRegressor(...ree=None, ...)"
,param_distributions,"{'colsample_bytree': [0.6, 0.7, ...], 'gamma': [0, 0.1, ...], 'learning_rate': [0.01, 0.03, ...], 'max_depth': [3, 4, ...], ...}"
,n_iter,40
,scoring,'neg_mean_absolute_error'
,n_jobs,-1
,refit,True
,cv,3
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [33]:
# getting best model
best_xgb = random_search.best_estimator_

print("Best parameters:")
print(random_search.best_params_)


Best parameters:
{'subsample': 1.0, 'reg_lambda': 1.5, 'reg_alpha': 1.0, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 8, 'learning_rate': 0.03, 'gamma': 0, 'colsample_bytree': 1.0}


In [34]:
# evaluate on test data
y_pred = best_xgb.predict(X_test_processed)

metrics = {
    "MAE": mean_absolute_error(y_test, y_pred),
    "MSE": mean_squared_error(y_test, y_pred),
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "R2": r2_score(y_test, y_pred)
}

metrics


{'MAE': 89.2426941217811,
 'MSE': 23649.513088319727,
 'RMSE': np.float64(153.78398189772471),
 'R2': 0.6876647099426042}

here, we can see that others params are still similar like the single model that we trained, and only the mean absoulte error improved well!

so, this is how the RandomSearchCV finds the best model which is trained with best parameters on passed grid.